In [2]:
pip install mlflow


  Using cached mlflow-3.9.0-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_skinny-3.9.0-py3-none-any.whl.metadata (32 kB)
  Using cached mlflow_tracing-3.9.0-py3-none-any.whl.metadata (19 kB)
  Using cached flask_cors-6.0.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached alembic-1.18.4-py3-none-any.whl.metadata (7.2 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached huey-2.6.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached skops-0.13.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached databricks_sdk-0.88.0-py3-none-any.whl.metadata (40 kB)
  Using cached opentelemetry_api-1.39.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_proto-1.39.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.39.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached sqlparse-0.5.5-py3-none-any.whl.m

In [7]:
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

In [ ]:
# ──── Debug rapide ────
coef = pipeline.named_steps['lasso'].coef_
print("Type des coefficients :", type(coef))
print("Longueur coef :", len(coef))
print("Nombre de colonnes dans features :", len(features.columns))
print("\nQuelques exemples de coef :", coef[:10])

# Vérifie si tous les coef sont numériques
print("\nTous numériques ?", all(isinstance(x, (int, float, np.number)) for x in coef))

# Si tu vois False → problème confirmé

In [12]:
def train_lasso_model(df):
    """Entraîne Lasso avec logging MLflow complet"""
    
    # Préprocessing 
    features = df.select_dtypes(include=[np.number]).drop(['SalePrice', 'Id'], axis=1).fillna(0)
    y = np.log(df['SalePrice'])
    
    X_train, X_test, y_train, y_test = train_test_split(
        features, y, test_size=0.2, random_state=42
    )
    
    with mlflow.start_run(run_name="Lasso_Best"):
        # Hyperparamètres
        params = {
            "alpha": 0.001, 
            "max_iter": 10000,
            "random_state": 42,
            "selection": "random"
        }
        mlflow.log_params(params)
        
        # Pipeline 
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('lasso', Lasso(**params))
        ])
        
        # Entraînement
        pipeline.fit(X_train, y_train)
        
        # Métriques test set 
        y_pred = pipeline.predict(X_test)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        test_r2 = r2_score(y_test, y_pred)
        
        # Cross-validation 
        cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
        cv_rmse = -cv_scores.mean()
        
        # Logging métriques détaillées
        mlflow.log_metric("test_rmse", test_rmse)
        mlflow.log_metric("test_r2", test_r2)
        mlflow.log_metric("cv_rmse", cv_rmse)
        mlflow.log_metric("residuals_mean", np.mean(y_test - y_pred))
        mlflow.log_metric("residuals_median", np.median(y_test - y_pred))
        mlflow.log_metric("residuals_std", np.std(y_test - y_pred))
        
        # Statistiques résidus 
        residuals = y_test - y_pred
        mlflow.log_metric("residuals_mean_target", -0.004416)
        mlflow.log_metric("residuals_median_target", 0.002235)
        mlflow.log_metric("residuals_std_target", 0.122054)
        
        # Sauvegarde modèle + feature importante
        mlflow.sklearn.log_model(pipeline, "lasso_champion_pipeline")
        
        # Feature importante (top 10)
        feature_importance = pd.DataFrame({
            'feature': features.columns,
            'importance': pipeline.named_steps['lasso'].coef_
        })

        # Nettoyage & abs
        feature_importance['importance'] = pd.to_numeric(feature_importance['importance'], errors='coerce')
        feature_importance = feature_importance.dropna(subset=['importance'])  # enlève les non-numériques
        feature_importance['importance'] = feature_importance['importance'].abs()
        feature_importance = feature_importance.nlargest(10, 'importance')
        
        feature_importance.to_csv('feature_importance.csv', index=False)
        mlflow.log_artifact('feature_importance.csv')
        
        print(f"🏆 Lasso Champion - Test RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}")
        print(f"   CV RMSE: {cv_rmse:.4f}")
        print(f"   Résidus - Moy: {np.mean(residuals):.6f}, Med: {np.median(residuals):.6f}, Std: {np.std(residuals):.6f}")
        
        return pipeline, test_rmse

In [13]:
if __name__ == "__main__":
    df = pd.read_csv('../data/train.csv')
    model, rmse = train_lasso_model(df)

2026/02/12 22:18:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\cheic\anaconda3\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏆 Lasso Champion - Test RMSE: 0.1509, R²: 0.8780
   CV RMSE: 0.1579
   Résidus - Moy: 0.001492, Med: 0.004154, Std: 0.150895
